In [175]:
import pandas as pd
import seaborn as sns
import random
import numpy as np
import math
penguins = pd.read_csv('penguins.csv')

In [176]:
penguins.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,FEMALE
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,MALE
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN


In [177]:
class ProcessingData:
    
    @staticmethod
    def shuffle(base):
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalize(base):
        base.iloc[:,2:-1] = (base.iloc[:,2:-1]-base.iloc[:,2:-1].min())/(base.iloc[:,2:-1].max()-base.iloc[:,2:-1].min())

    @staticmethod
    def devide(base):
        n = int(len(base)*0.7)
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

In [178]:
penguinsEdit = penguins.copy()

ProcessingData.shuffle(penguinsEdit)
ProcessingData.normalize(penguinsEdit)
trening_data,validate_data = ProcessingData.devide(penguinsEdit)

In [179]:
class SoftSet:
    
    @staticmethod
    def recoverTreningData(data):
        sortedData = []
        sort = data.groupby(data.species)
        sortedData.append(sort.get_group("Adelie"))
        sortedData.append(sort.get_group("Chinstrap"))
        sortedData.append(sort.get_group("Gentoo"))

        return sortedData

    @staticmethod
    def meanMaker(sortedData):
        meanTable = []
        for j in range(len(sortedData)):
            meanTable.append(sortedData[j].mean(numeric_only=True).tolist())
        return meanTable

    @staticmethod
    def make10Table(data,meanTable):
        oneZeroTable = []
        for j in range(len(data)):
            speciesTable = []
            for k in range(2,len(data[j].columns)-1):
                smaller = data[j][data[j].iloc[:,k] < meanTable[j][k-2]].count()
                bigger = len(data[j]) - smaller
                
                if smaller[k] > bigger[k]:
                    speciesTable.extend([1,0])
                else:
                     speciesTable.extend([0,1])
            oneZeroTable.append(speciesTable)
        return oneZeroTable

    @staticmethod
    def softSet(validate_data,oneZeroTable,meanTable):
        correct = 0
        varieties = ["Adelie","Chinstrap","Gentoo"]
        for i in range(len(validate_data)):
            trueName = validate_data.iloc[i].species
            oneZeroValidate = []
            for j in range(len(meanTable)):
                oneZeroValidateOne = []
                for k in range(2,len(validate_data.iloc[i])-1):
                    if validate_data.iloc[i][k] < meanTable[j][k-2]:
                        oneZeroValidateOne.extend([1,0])
                    else:
                        oneZeroValidateOne.extend([0,1])
                oneZeroValidate.append(oneZeroValidateOne)
                 
            tableOfSums = []
            for j in range(len(oneZeroValidate)):
                suma = 0
                for sum in range(len(oneZeroValidate[j])):
                    suma+=oneZeroValidate[j][sum]*oneZeroTable[j][sum]
                tableOfSums.append(suma)
                

            guessedName = varieties[np.argmax(tableOfSums)]

            if guessedName == trueName:
                correct+=1
        return correct,correct/len(validate_data)*100

In [180]:
sortedData = SoftSet.recoverTreningData(trening_data)
meanTable = SoftSet.meanMaker(sortedData)
oneZeroTable = SoftSet.make10Table(sortedData,meanTable)
correct, accuracy = SoftSet.softSet(validate_data,oneZeroTable,meanTable)
print("{}/{} accuracy => {:.2f}%".format(correct,len(validate_data),accuracy))

47/104 accuracy => 45.19%
